In [5]:
from win32com.client import Dispatch
import cv2
import numpy as np
import time
from datetime import datetime
import os
import csv
import joblib

In [6]:
model=joblib.load('./Models/detect.joblib')
vals=joblib.load('./Models/vals')

In [9]:
def speak(str1):
    speaker = Dispatch("SAPI.SpVoice")
    speaker.Speak(str1)

facedetect = cv2.CascadeClassifier(r'C:\Users\user\anaconda3\Lib\site-packages\cv2\data\haarcascade_frontalface_default.xml')

video = cv2.VideoCapture(0)

print('Capturing.....')

COL_NAMES = ['NAME', 'TIME']

while True:
    ret, frame = video.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = facedetect.detectMultiScale(gray, 1.3, 5)
    
    for (x, y, w, h) in faces:
        crop_img = frame[y:y+h, x:x+w, :]
        resized_img = cv2.resize(crop_img, (128, 128))
        arr_img = np.array(resized_img)
        reshaped_img = arr_img.reshape(-1, 128, 128, 3)
        
        output = np.argmax(model.predict([reshaped_img]))
        
        if output < len(vals):
            name = vals[int(output)]
            ts = time.time()
            date = datetime.fromtimestamp(ts).strftime("%d-%m-%Y")
            timestamp = datetime.fromtimestamp(ts).strftime("%H:%M:%S")
            exist = os.path.isfile("Attendance/Attendance_" + date + ".csv")
            
            cv2.putText(frame, str(name), (x, y), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 0), 4)
            
            attendance = [str(name), str(timestamp)]
            
    cv2.imshow("Frame", frame)
    k = cv2.waitKey(1)
    
    if k == ord('o'):
        speak('Attendance Taken..')
        
        with open("Attendance/Attendance_" + date + ".csv", "a") as csvfile:
            writer = csv.writer(csvfile)
            
            if exist:
                writer.writerow(attendance)
            else:
                writer.writerow(COL_NAMES)
                writer.writerow(attendance)
    
    if k == ord('q'):
        break

video.release()
cv2.destroyAllWindows()


Capturing.....
1/1 [==============================] - 0s 55ms/step


1/1 [==============================] - 0s 55ms/step


1/1 [==============================] - 0s 35ms/step


1/1 [==============================] - 0s 40ms/step


In [4]:
vals

{0: 'Ajmal', 1: 'Anjana', 2: 'Ashiqu', 3: 'Azra'}